## User Revoke process 
user email parameter is created for revoking all the token associate to that use If user email is empty it wont revoke any user token just monitor token the process

In [0]:
# Input parameter: User email
dbutils.widgets.text("user_email", "", "User Email")
user_email = dbutils.widgets.get("user_email")

## Databricks Token Monitoring Pcoess Started 
   Below code will cosider all the token created on current workspace and create tmp view

In [0]:
from databricks.sdk import WorkspaceClient
w = WorkspaceClient()
spark.createDataFrame([token.as_dict() for token in w.token_management.list()]).createOrReplaceTempView('tokens')
display(spark.sql('select * from tokens order by creation_time'))

Retrive the workspace id for auditing purpose to get know the worksapce details if we are plaining to keep all token at centarl location for reporting. 

In [0]:
context = dbutils.notebook.entry_point.getDbutils().notebook().getContext()
workspace_id = context.workspaceId().get()  # Retrieve the Workspace ID directly
print(f"Workspace ID: {workspace_id}")

Workspace ID: 1102558365765630


In [0]:
Query = f"""
select 
{workspace_id} AS workspace_id,
created_by_id,
created_by_username,
owner_id,token_id,
from_unixtime(creation_time/1000) AS creation_datetime ,
from_unixtime(expiry_time/1000)  AS expire_datetime,
comment
 from tokens """

In [0]:
spark.sql(Query).createOrReplaceTempView("databricks_tokens")

Delete And Insert Process in to Monitoring table 

In [0]:
del_query = f"DELETE from monitor.logging.databricks_tokens where workspace_id = '{workspace_id}'"
spark.sql(del_query)

DataFrame[num_affected_rows: bigint]

In [0]:
%sql
INSERT INTO monitor.logging.databricks_tokens 
select 
workspace_id,
created_by_id,
created_by_username,
owner_id,
token_id,
creation_datetime ,
expire_datetime,
comment
 from databricks_tokens 

num_affected_rows,num_inserted_rows
12,12


### User Email as parameter and revoke the tokens

In [0]:
df = spark.sql(f"select * from monitor.logging.databricks_tokens where created_by_username='{user_email}'")

In [0]:
from databricks.sdk import WorkspaceClient

for df in spark.sql(
    f"select * from monitor.logging.databricks_tokens where created_by_username='{user_email}'"
).collect():
    print(df['token_id'])
    if df['token_id'] is not None:
        print("token delete for user", df['created_by_username'])
        token_id = df['token_id']
        w = WorkspaceClient()
        try:
            w.token_management.delete(token_id=f'{token_id}')
        except Exception as e:
            if "ResourceDoesNotExist" in str(e):
                print(f"Token with id {token_id} does not exist")
            else:
                raise e
    else:
        print(f"No token found for user, {df['created_by_username']}")

In [0]:
print('Successfully deleted tokens')

Successfully deleted tokens
